In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
import sys
sys.path.append('..')
from dataset import batcher

In [9]:
MODEL_DIR = '../saved_models/grid_search_all/'
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']
# COUNTRIES = ['ethiopia-2018', 'nigeria-2018', 'malawi-2016']
COUNTRIES = ['ethiopia-2018', 'nigeria-2018', 'malawi-2016', 'pollution-2018']

In [10]:
# Get test split
bands = ['BLUE', 'GREEN', 'RED', 'NIR', 'SW_IR1', 'SW_IR2', 'TEMP', 'VIIRS', 'DELTA_TEMP', 'CO']
# bands = ['VIIRS']
def batch(country_year):
    test_batcher = batcher.Batcher(bands, country_year=country_year, bucket=False, shuffle=False, split='custom', repeat=1, batch_size=3000).get_dataset()
    x_true = []
    y_true = []
    for x, y in test_batcher:
        x_true.append(x)
        y_true.append(y)

    y_true = np.squeeze(y_true)
    return x_true, y_true

In [11]:
# Load model
def load_model_from_dir(model_name):
    model_path = MODEL_DIR + model_name + '.h5'
    model = tf.keras.models.load_model(model_path, compile=False)
    
    return model

In [71]:
# Model, Learning Rate Grid Search - All bands
epochs = [10, 50, 100, 200]
lrs = [1e-1, 1e-2, 1e-3, 1e-4]

def run_epoch_lr_grid_search(country_year):
    grid_search_results = pd.DataFrame(columns=['experiment', 'epoch', 'learning_rate', 'r2'])
    x_true, y_true = batch(country_year)

    for e in epochs:
        for l in lrs:
            experiment_name='imagery_sample_cnn_regression' + str(e) + '_' + str(l)

            # Set hyperparamters
            opt = tf.keras.optimizers.Adam(learning_rate=l)

            # Get consumption predictions
            trained_model = load_model_from_dir(experiment_name)
            trained_model.compile(
                        loss=LOSS,
                        optimizer=opt,
                        metrics=METRICS,
            )

            y_preds = trained_model.predict_on_batch(x_true)
            y_preds = np.squeeze(y_preds)
            r2 = pearsonr(y_true, y_preds)[0]

            results = {'experiment': experiment_name, 'epoch': e, 'learning_rate': l, 'r2': r2}
            grid_search_results = grid_search_results.append(results, ignore_index = True)
            
    return grid_search_results

for country_year in COUNTRIES:  
    country_grid_search_results = run_epoch_lr_grid_search(country_year)
    print(f'{country_year}:')
    print(country_grid_search_results.sort_values(by=['r2']))

2023-02-24 11:48:32.901070: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/2970009259.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-24 11:48:35.985977: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/2970009259.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-24 11:48:38.705176: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.

ethiopia-2018:
                                 experiment epoch  learning_rate        r2
8      imagery_sample_cnn_regression100_0.1   100         0.1000 -0.097972
6     imagery_sample_cnn_regression50_0.001    50         0.0010 -0.087158
12     imagery_sample_cnn_regression200_0.1   200         0.1000 -0.033941
14   imagery_sample_cnn_regression200_0.001   200         0.0010 -0.002827
9     imagery_sample_cnn_regression100_0.01   100         0.0100 -0.002788
13    imagery_sample_cnn_regression200_0.01   200         0.0100  0.000570
11  imagery_sample_cnn_regression100_0.0001   100         0.0001  0.000735
3    imagery_sample_cnn_regression10_0.0001    10         0.0001  0.002891
5      imagery_sample_cnn_regression50_0.01    50         0.0100  0.010881
4       imagery_sample_cnn_regression50_0.1    50         0.1000  0.013716
7    imagery_sample_cnn_regression50_0.0001    50         0.0001  0.038972
10   imagery_sample_cnn_regression100_0.001   100         0.0010  0.043488
0       im

2023-02-24 11:49:19.804110: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/2970009259.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-24 11:49:22.720009: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/2970009259.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-24 11:49:25.541762: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.

nigeria-2018:
                                 experiment epoch  learning_rate        r2
4       imagery_sample_cnn_regression50_0.1    50         0.1000 -0.372287
0       imagery_sample_cnn_regression10_0.1    10         0.1000 -0.366484
3    imagery_sample_cnn_regression10_0.0001    10         0.0001 -0.319038
10   imagery_sample_cnn_regression100_0.001   100         0.0010 -0.173427
1      imagery_sample_cnn_regression10_0.01    10         0.0100 -0.146123
6     imagery_sample_cnn_regression50_0.001    50         0.0010 -0.143286
12     imagery_sample_cnn_regression200_0.1   200         0.1000 -0.126546
11  imagery_sample_cnn_regression100_0.0001   100         0.0001 -0.120490
5      imagery_sample_cnn_regression50_0.01    50         0.0100  0.026656
14   imagery_sample_cnn_regression200_0.001   200         0.0010  0.071014
13    imagery_sample_cnn_regression200_0.01   200         0.0100  0.083409
2     imagery_sample_cnn_regression10_0.001    10         0.0010  0.128012
7    imager

/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/2970009259.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)


KeyError: in user code:

    File "/Users/omshah/PovertyPredictionProject/Predicting-Poverty-Ethiopia/model_analysis/../dataset/batcher.py", line 127, in to_tuple  *
        band = (band - dc.MEANS_DICT[self.country_year][key]) / dc.STD_DEVS_DICT[self.country_year][key]

    KeyError: 'CO'


In [56]:
# Model, Learning Rate Grid Search - All bands
models = ['sample_cnn', 'deep_sample_cnn', 'sample_vgg']
lrs = [1e-1, 1e-2, 1e-3, 1e-4]

def run_model_lr_grid_search(country_year):
    grid_search_results = pd.DataFrame(columns=['experiment', 'model', 'learning_rate', 'r2'])
    x_true, y_true = batch(country_year)

    for m in models:
        for l in lrs:
            experiment_name='imagery_' + m + '_regression' + '_' + str(l)

            # Set hyperparamters
            opt = tf.keras.optimizers.Adam(learning_rate=l)

            # Get consumption predictions
            trained_model = load_model_from_dir(experiment_name)
            trained_model.compile(
                        loss=LOSS,
                        optimizer=opt,
                        metrics=METRICS,
            )

            y_preds = trained_model.predict_on_batch(x_true)
            y_preds = np.squeeze(y_preds)
            r2 = pearsonr(y_true, y_preds)[0]

            results = {'experiment': experiment_name, 'model': m, 'learning_rate': l, 'r2': r2}
            grid_search_results = grid_search_results.append(results, ignore_index = True)
            
    return grid_search_results

for country_year in COUNTRIES:  
    country_grid_search_results = run_lr_model_grid_search(country_year)
    print(f'{country_year}:')
    print(country_grid_search_results.sort_values(by=['r2']))

In [12]:
# Batch size, Learning Rate, Activation Function Grid Search - All bands
batch_sizes = [32, 64, 128, 256]
lrs = [1e-1, 1e-2, 1e-3, 1e-4]
activations = ['relu', 'gelu', 'tanh']

def run_bs_lr_activation_grid_search(country_year):
    grid_search_results = pd.DataFrame(columns=['experiment', 'batch_size', 'learning_rate', 'activation', 'r2'])
    x_true, y_true = batch(country_year)

    for b in batch_sizes:
        for l in lrs:
            for a in activations:
                experiment_name = 'imagery_sample_cnn_regression' + '_' + str(b) + str(l) + a

                # Set hyperparamters
                opt = tf.keras.optimizers.Adam(learning_rate=l)

                # Get consumption predictions
                trained_model = load_model_from_dir(experiment_name)
                trained_model.compile(
                            loss=LOSS,
                            optimizer=opt,
                            metrics=METRICS,
                )

                y_preds = trained_model.predict_on_batch(x_true)
                y_preds = np.squeeze(y_preds)
                r2 = pearsonr(y_true, y_preds)[0]

                results = {'experiment': experiment_name, 'batch_size': b, 'learning_rate': l, 'activation': a, 'r2': r2}
                grid_search_results = grid_search_results.append(results, ignore_index = True)
            
    return grid_search_results

for country_year in COUNTRIES:  
    country_grid_search_results = run_bs_lr_activation_grid_search(country_year)
    print(f'{country_year}:')
    display(country_grid_search_results.sort_values(by=['r2']))

2023-05-27 11:48:53.080127: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-27 11:48:53.080358: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-05-27 11:48:53.257685: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Metal device set to: Apple M1 Pro


2023-05-27 11:48:57.882733: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_64411/3718368320.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-05-27 11:49:02.579973: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_64411/3718368320.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-05-27 11:49:07.755374: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registr

/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_64411/3718368320.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-05-27 11:49:18.722639: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_64411/3718368320.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-05-27 11:49:21.974143: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_64411/3718368320.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-05-27 11:49:25.558877: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_64411/3718368320.py:31: FutureWarning

...

/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_64411/3718368320.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-05-27 11:59:34.233133: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_64411/3718368320.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-05-27 11:59:39.691148: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_64411/3718368320.py:31: FutureWarning

2023-05-27 12:00:38.011013: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_64411/3718368320.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-05-27 12:00:41.166130: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_64411/3718368320.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-05-27 12:00:46.595735: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registr

pollution-2018:


/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_64411/3718368320.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)


,experiment,batch_size,learning_rate,activation,r2
2,imagery_sample_cnn_regression_320.1tanh,32,0.1000,tanh,-0.148531
11,imagery_sample_cnn_regression_320.0001tanh,32,0.0001,tanh,-0.115745
9,imagery_sample_cnn_regression_320.0001relu,32,0.0001,relu,-0.088506
10,imagery_sample_cnn_regression_320.0001gelu,32,0.0001,gelu,-0.075124
16,imagery_sample_cnn_regression_640.01gelu,64,0.0100,gelu,-0.072534
34,imagery_sample_cnn_regression_1280.0001gelu,128,0.0001,gelu,-0.062997
40,imagery_sample_cnn_regression_2560.01gelu,256,0.0100,gelu,-0.044083
12,imagery_sample_cnn_regression_640.1relu,64,0.1000,relu,-0.037335
28,imagery_sample_cnn_regression_1280.01gelu,128,0.0100,gelu,-0.034892
0,imagery_sample_cnn_regression_320.1relu,32,0.1000,relu,-0.027038


In [ ]:
"""
Ethiopia: (4) 32, GELU, 0.01 potential: 43, 3
Nigeria: (6) 32, RELU, 0.001 potential: 43
Malawi: (43) 256, GELU, 0.001 potential: 3, 6
Pooled: (3) 32, RELU, 0.01 potential: 6, 43

Likely Best: 256, GELU, 0.001
"""